In [1]:
"""
WEEK 8 - DETAILED CODE FOR EACH FUNCTION
Shows exactly how each input was generated using ML models
"""

import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel
from scipy.optimize import minimize
from scipy.stats import norm

print("="*80)
print("WEEK 8 ML CODE - FUNCTION BY FUNCTION")
print("="*80)
print()

# ============================================================================
# F1 (2D) - RECOVERY: Upper Confidence Bound
# ============================================================================
print("="*80)
print("F1 (2D) - RECOVERY MISSION")
print("="*80)
print()

# Historical data (7 weeks)
X_f1 = np.array([
    [0.10, 0.10],   # Week 1: 0.0
    [0.12, 0.08],   # Week 2: 0.0
    [0.21, 0.11],   # Week 3: 0.0
    [0.14, 0.14],   # Week 4: 0.0
    [0.08, 0.08],   # Week 5: 0.0
    [0.45, 0.45],   # Week 6: 0.0128 ✓ SIGNAL!
    [0.48, 0.48],   # Week 7: 0.000008 (lost it)
])
y_f1 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0128, 0.000008])

print("CODE:")
print("-"*80)
print("""
# Build GP model
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1, 0.1])
gp_f1 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=False
)
gp_f1.fit(X_f1, y_f1)

# Test candidates (explore LOWER than Week 6)
candidates = {
    "Lower both": [0.42, 0.42],
    "Much lower": [0.40, 0.40],
    "Asymmetric 1": [0.43, 0.40],
    "Asymmetric 2": [0.40, 0.43],
}

# Select using Upper Confidence Bound (optimistic for recovery)
best_ucb = -np.inf
for name, cand in candidates.items():
    mu, std = gp_f1.predict([cand], return_std=True)
    ucb = mu[0] + 2*std[0]  # Optimistic
    if ucb > best_ucb:
        best_ucb = ucb
        week8_f1 = cand
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1, 0.1])
gp_f1 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15, 
                                 alpha=1e-6, normalize_y=False)
gp_f1.fit(X_f1, y_f1)

candidates = {
    "Lower both": [0.42, 0.42],
    "Much lower": [0.40, 0.40],
    "Asymmetric 1": [0.43, 0.40],
    "Asymmetric 2": [0.40, 0.43],
}

print("\nRESULTS:")
print("-"*80)
best_ucb = -np.inf
best_input = None
for name, cand in candidates.items():
    mu, std = gp_f1.predict([cand], return_std=True)
    ucb = mu[0] + 2*std[0]
    print(f"{name:20s}: μ={mu[0]:.6f}, σ={std[0]:.6f}, UCB={ucb:.6f}")
    if ucb > best_ucb:
        best_ucb = ucb
        best_input = cand

week8_f1 = np.array(best_input)
print(f"\n✓ SELECTED: {week8_f1}")
print(f"  Rationale: Highest UCB (optimistic for recovery)")
print(f"  Week 7 went [0.45→0.48] and lost signal → try LOWER")
print()

# ============================================================================
# F2 (2D) - CRITICAL: Exploitation near narrow peak
# ============================================================================
print("="*80)
print("F2 (2D) - CRITICAL RECOVERY")
print("="*80)
print()

X_f2 = np.array([
    [0.10, 0.10],   # Week 1: 0.0892
    [0.12, 0.08],   # Week 2: 0.0705
    [0.21, 0.11],   # Week 3: 0.0295
    [0.14, 0.14],   # Week 4: 0.0150
    [0.08, 0.08],   # Week 5: 0.0463
    [0.111, 0.100], # Week 6: 0.1300 ✓ BEST!
    [0.11, 0.10],   # Week 7: 0.0468 (disaster)
])
y_f2 = np.array([0.0892, 0.0705, 0.0295, 0.0150, 0.0463, 0.1300, 0.0468])

print("CODE:")
print("-"*80)
print("""
# Build GP with VERY small length scale (narrow peak!)
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.02, 0.02])
gp_f2 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-7,  # Very small noise
    normalize_y=True
)
gp_f2.fit(X_f2, y_f2)

# Test candidates VERY close to Week 6 peak
candidates = {
    "Near Week 6 (1)": [0.1105, 0.1005],
    "Near Week 6 (2)": [0.1115, 0.0995],
    "Near Week 6 (3)": [0.1110, 0.1000],
    "Slightly higher": [0.1120, 0.1010],
    "Dim 1 focus": [0.1125, 0.1000],
}

# Select best predicted mean (pure exploitation)
best_mean = -np.inf
for name, cand in candidates.items():
    mu, std = gp_f2.predict([cand], return_std=True)
    if mu[0] > best_mean:
        best_mean = mu[0]
        week8_f2 = cand
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.02, 0.02])
gp_f2 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-7, normalize_y=True)
gp_f2.fit(X_f2, y_f2)

candidates = {
    "Near Week 6 (1)": [0.1105, 0.1005],
    "Near Week 6 (2)": [0.1115, 0.0995],
    "Near Week 6 (3)": [0.1110, 0.1000],
    "Slightly higher": [0.1120, 0.1010],
    "Dim 1 focus": [0.1125, 0.1000],
}

print("\nRESULTS:")
print("-"*80)
best_mean = -np.inf
best_input = None
for name, cand in candidates.items():
    mu, std = gp_f2.predict([cand], return_std=True)
    print(f"{name:20s}: μ={mu[0]:.6f}, σ={std[0]:.6f}")
    if mu[0] > best_mean:
        best_mean = mu[0]
        best_input = cand

week8_f2 = np.array(best_input)
print(f"\n✓ SELECTED: {week8_f2}")
print(f"  Predicted: {best_mean:.6f}")
print(f"  Rationale: Maximize mean prediction (exploitation)")
print(f"  Stay VERY close to Week 6 peak (ultra-narrow)")
print()

# ============================================================================
# F3 (3D) - DAMAGE CONTROL: Return to best
# ============================================================================
print("="*80)
print("F3 (3D) - DAMAGE CONTROL")
print("="*80)
print()

X_f3 = np.array([
    [0.80, 0.80, 0.80],
    [0.95, 0.95, 0.95],
    [0.98, 0.99, 0.87],
    [0.948885, 0.965632, 0.808397],  # Week 4: -0.0786 BEST
    [1.01, 1.01, 0.82],
    [0.928, 0.832, 0.004],
    [0.99, 0.99, 0.99],  # Week 7: -0.427 (disaster)
])
y_f3 = np.array([-0.1055, -0.0919, -0.0856, -0.0786, -1.1543, -0.1161, -0.427251295])

print("CODE:")
print("-"*80)
print("""
# Build GP with Matérn kernel (sharp boundaries)
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.3, 0.3, 0.3], nu=2.5)
gp_f3 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)
gp_f3.fit(X_f3, y_f3)

# Test candidates near Week 4 best
candidates = {
    "Near Week 4": [0.95, 0.97, 0.81],
    "Slight variation": [0.94, 0.96, 0.80],
    "Conservative": [0.93, 0.95, 0.79],
}

# Select least negative (closest to 0)
best_val = -np.inf
for name, cand in candidates.items():
    mu, std = gp_f3.predict([cand], return_std=True)
    if mu[0] > best_val:  # Want least negative
        best_val = mu[0]
        week8_f3 = cand
""")

# Actually run it
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.3, 0.3, 0.3], nu=2.5)
gp_f3 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f3.fit(X_f3, y_f3)

candidates = {
    "Near Week 4": [0.95, 0.97, 0.81],
    "Slight variation": [0.94, 0.96, 0.80],
    "Conservative": [0.93, 0.95, 0.79],
}

print("\nRESULTS:")
print("-"*80)
best_val = -np.inf
best_input = None
for name, cand in candidates.items():
    mu, std = gp_f3.predict([cand], return_std=True)
    print(f"{name:20s}: μ={mu[0]:.6f}, σ={std[0]:.6f}")
    if mu[0] > best_val:
        best_val = mu[0]
        best_input = cand

week8_f3 = np.array(best_input)
print(f"\n✓ SELECTED: {week8_f3}")
print(f"  Predicted: {best_val:.6f}")
print(f"  Rationale: Return near Week 4 best, minimize damage")
print()

# ============================================================================
# F4 (4D) - DAMAGE CONTROL: Stay at center
# ============================================================================
print("="*80)
print("F4 (4D) - DAMAGE CONTROL")
print("="*80)
print()

X_f4 = np.array([
    [0.5, 0.5, 0.5, 0.5],  # Week 1: -3.986 BEST
    [0.3, 0.3, 0.3, 0.3],
    [0.44, 0.29, 0.35, 1.25],
    [0.51, 0.60, 0.57, 0.01],
    [0.66, 0.30, 0.30, 0.36],
    [0.2, 0.2, 0.95, 0.4],
    [0.65, 0.65, 0.65, 0.65],  # Week 7: -15.16
])
y_f4 = np.array([-3.986, -4.306, -30.129, -12.492, -7.262, -19.009, -15.158011980])

print("CODE:")
print("-"*80)
print("""
# Build GP with Matérn kernel
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.2]*4, nu=2.5)
gp_f4 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)
gp_f4.fit(X_f4, y_f4)

# Test candidates near center [0.5, 0.5, 0.5, 0.5]
candidates = {
    "Near center (1)": [0.505, 0.495, 0.500, 0.500],
    "Near center (2)": [0.495, 0.505, 0.500, 0.500],
    "Slight asymmetry": [0.52, 0.48, 0.50, 0.50],
}

# Select least negative
best_val = -np.inf
for name, cand in candidates.items():
    mu, std = gp_f4.predict([cand], return_std=True)
    if mu[0] > best_val:
        best_val = mu[0]
        week8_f4 = cand
""")

# Actually run it
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.2]*4, nu=2.5)
gp_f4 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f4.fit(X_f4, y_f4)

candidates = {
    "Near center (1)": [0.505, 0.495, 0.500, 0.500],
    "Near center (2)": [0.495, 0.505, 0.500, 0.500],
    "Slight asymmetry": [0.52, 0.48, 0.50, 0.50],
}

print("\nRESULTS:")
print("-"*80)
best_val = -np.inf
best_input = None
for name, cand in candidates.items():
    mu, std = gp_f4.predict([cand], return_std=True)
    print(f"{name:20s}: μ={mu[0]:.6f}, σ={std[0]:.6f}")
    if mu[0] > best_val:
        best_val = mu[0]
        best_input = cand

week8_f4 = np.array(best_input)
print(f"\n✓ SELECTED: {week8_f4}")
print(f"  Predicted: {best_val:.6f}")
print(f"  Rationale: Stay very close to center (confirmed optimal)")
print()

# ============================================================================
# F5 (4D) - EXPLOITATION: Local search on sharp peak
# ============================================================================
print("="*80)
print("F5 (4D) - EXPLOIT SUCCESS 🚀")
print("="*80)
print()

X_f5 = np.array([
    [0.30, 0.30, 0.30, 0.30],
    [0.28, 0.32, 0.30, 0.29],
    [0.344822, 0.264687, 0.374156, 0.203902],
    [0.196828, 0.320017, 0.300, 0.289958],
    [0.99, 0.90, 0.98, 0.93],  # Week 5: 5549
    [0.985, 0.905, 0.975, 0.925],  # Week 6: 5399
    [1.0, 0.853, 1.0, 0.977],  # Week 7: 6158 NEW BEST!
])
y_f5 = np.array([136.85, 137.29, 131.78, 140.74, 5549.45, 5398.58, 6158.076608789])

print("CODE:")
print("-"*80)
print("""
# Build GP with small length scale (sharp peak)
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.05, 0.05, 0.05, 0.05])
gp_f5 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f5.fit(X_f5, y_f5)

# Local search using gradient descent
def neg_pred(x):
    return -gp_f5.predict([x])[0]

# Multiple restarts near Week 7 best
starts = [
    [1.0, 0.85, 1.0, 0.98],
    [0.995, 0.855, 1.0, 0.975],
    [1.0, 0.850, 0.995, 0.980],
    [0.998, 0.860, 1.0, 0.970],
]

best_val = -np.inf
for start in starts:
    result = minimize(
        neg_pred, 
        start, 
        method='L-BFGS-B',
        bounds=[(0.85, 1.0), (0.80, 0.90), (0.95, 1.0), (0.95, 1.0)]
    )
    mu = gp_f5.predict([result.x])[0]
    if mu > best_val:
        best_val = mu
        week8_f5 = result.x
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.05, 0.05, 0.05, 0.05])
gp_f5 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=True)
gp_f5.fit(X_f5, y_f5)

def neg_pred(x):
    return -gp_f5.predict([x])[0]

starts = [
    [1.0, 0.85, 1.0, 0.98],
    [0.995, 0.855, 1.0, 0.975],
    [1.0, 0.850, 0.995, 0.980],
    [0.998, 0.860, 1.0, 0.970],
]

print("\nRESULTS:")
print("-"*80)
best_val = -np.inf
best_input = None
for i, start in enumerate(starts):
    result = minimize(neg_pred, start, method='L-BFGS-B',
                     bounds=[(0.85, 1.0), (0.80, 0.90), (0.95, 1.0), (0.95, 1.0)])
    mu = gp_f5.predict([result.x])[0]
    print(f"Start {i+1}: {start} → {result.x} → μ={mu:.2f}")
    if mu > best_val:
        best_val = mu
        best_input = result.x

week8_f5 = np.array(best_input)
print(f"\n✓ SELECTED: {week8_f5}")
print(f"  Predicted: {best_val:.2f}")
print(f"  Rationale: L-BFGS-B local search around Week 7 peak")
print()

# ============================================================================
# F6 (5D) - DAMAGE CONTROL: Return to Week 5
# ============================================================================
print("="*80)
print("F6 (5D) - DAMAGE CONTROL")
print("="*80)
print()

X_f6 = np.array([
    [0.75, 0.75, 0.75, 0.75, 0.75],
    [0.3, 0.3, 0.3, 0.3, 0.3],
    [0.49, 0.02, 0.45, 0.40, 0.32],
    [0.69, 0.001, 0.04, 0.001, 0.001],
    [0.26, 0.18, 0.50, 0.48, 0.41],  # Week 5: -1.092 BEST
    [0.1, 0.1, 0.7, 0.7, 0.6],
    [0.15, 0.15, 0.50, 0.50, 0.70],  # Week 7: -1.552
])
y_f6 = np.array([-1.521, -1.139, -1.123, -2.067, -1.092, -1.231, -1.551724443])

print("CODE:")
print("-"*80)
print("""
# Build GP with RBF kernel
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*5)
gp_f6 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)
gp_f6.fit(X_f6, y_f6)

# Test candidates near Week 5 best
candidates = {
    "Near Week 5 (1)": [0.25, 0.17, 0.50, 0.49, 0.42],
    "Near Week 5 (2)": [0.27, 0.19, 0.51, 0.47, 0.40],
    "Dim 5 moderate": [0.26, 0.18, 0.50, 0.48, 0.45],
}

# Select least negative
best_val = -np.inf
for name, cand in candidates.items():
    mu, std = gp_f6.predict([cand], return_std=True)
    if mu[0] > best_val:
        best_val = mu[0]
        week8_f6 = cand
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*5)
gp_f6 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f6.fit(X_f6, y_f6)

candidates = {
    "Near Week 5 (1)": [0.25, 0.17, 0.50, 0.49, 0.42],
    "Near Week 5 (2)": [0.27, 0.19, 0.51, 0.47, 0.40],
    "Dim 5 moderate": [0.26, 0.18, 0.50, 0.48, 0.45],
}

print("\nRESULTS:")
print("-"*80)
best_val = -np.inf
best_input = None
for name, cand in candidates.items():
    mu, std = gp_f6.predict([cand], return_std=True)
    print(f"{name:20s}: μ={mu[0]:.6f}, σ={std[0]:.6f}")
    if mu[0] > best_val:
        best_val = mu[0]
        best_input = cand

week8_f6 = np.array(best_input)
print(f"\n✓ SELECTED: {week8_f6}")
print(f"  Predicted: {best_val:.6f}")
print(f"  Rationale: Return near Week 5 best (Dim 5 push failed)")
print()

# ============================================================================
# F7 (6D) - EXPLOITATION: Maximize mean
# ============================================================================
print("="*80)
print("F7 (6D) - EXPLOIT SUCCESS")
print("="*80)
print()

X_f7 = np.array([
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
    [0.21, 0.19, 0.21, 0.19, 0.17, 0.19],
    [0.08, 0.32, 0.15, 0.28, 0.41, 0.27],
    [0.05, 0.50, 0.25, 0.20, 0.15, 0.85],
    [0.06, 0.48, 0.25, 0.20, 0.40, 0.75],
    [0.038, 0.462, 0.239, 0.171, 0.378, 0.734],  # Week 7: 1.478
])
y_f7 = np.array([0.000034, 0.408, 0.347, 0.568, 0.836, 1.435, 1.478289390])

print("CODE:")
print("-"*80)
print("""
# Build GP with RBF kernel
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1]*6)
gp_f7 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f7.fit(X_f7, y_f7)

# Local optimization (maximize mean)
def neg_pred(x):
    return -gp_f7.predict([x])[0]

result = minimize(
    neg_pred, 
    X_f7[-1],  # Start from Week 7
    method='L-BFGS-B',
    bounds=[(0.0, 0.1), (0.4, 0.5), (0.2, 0.3), 
            (0.1, 0.2), (0.3, 0.4), (0.7, 0.8)]
)

week8_f7 = result.x
predicted = gp_f7.predict([week8_f7])[0]
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1]*6)
gp_f7 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=True)
gp_f7.fit(X_f7, y_f7)

def neg_pred_f7(x):
    return -gp_f7.predict([x])[0]

result = minimize(neg_pred_f7, X_f7[-1], method='L-BFGS-B',
                 bounds=[(0.0, 0.1), (0.4, 0.5), (0.2, 0.3), 
                        (0.1, 0.2), (0.3, 0.4), (0.7, 0.8)])

week8_f7 = result.x
mu_f7 = gp_f7.predict([week8_f7])[0]

print("\nRESULTS:")
print("-"*80)
print(f"Week 7 input: {X_f7[-1]}")
print(f"Optimized:    {week8_f7}")
print(f"Predicted:    {mu_f7:.6f}")
print(f"\n✓ SELECTED: {week8_f7}")
print(f"  Rationale: L-BFGS-B maximization of predicted mean")
print()

# ============================================================================
# F8 (8D) - BALANCED: Expected Improvement
# ============================================================================
print("="*80)
print("F8 (8D) - MAINTAIN PERFORMANCE")
print("="*80)
print()

X_f8 = np.array([
    [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    [0.12, 0.09, 0.11, 0.10, 0.08, 0.13, 0.11, 0.09],
    [0.29, 0.25, 0.02, 0.29, 0.14, 0.22, 0.25, 0.30],
    [1.0, 0.001, 1.0, 0.001, 0.001, 1.0, 1.0, 0.001],
    [0.05, 0.25, 0.25, 0.25, 0.25, 0.25, 0.05, 0.05],
    [0.18, 0.15, 0.20, 0.15, 0.25, 0.15, 0.15, 0.18],
    [0.177, 0.194, 0.170, 0.194, 0.294, 0.143, 0.109, 0.208],  # Week 7: 9.692
])
y_f8 = np.array([9.542, 9.554, 9.548, 4.180, 9.643, 9.676, 9.692074600])

print("CODE:")
print("-"*80)
print("""
# Build GP with ARD kernel (8D feature selection)
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*8)
gp_f8 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f8.fit(X_f8, y_f8)

# Expected Improvement acquisition function
best_y = np.max(y_f8)

def expected_improvement(x):
    mu, std = gp_f8.predict([x], return_std=True)
    mu = mu[0]
    std = std[0]
    if std == 0:
        return 0
    improvement = mu - best_y - 0.01  # Small epsilon
    Z = improvement / std
    ei = improvement * norm.cdf(Z) + std * norm.pdf(Z)
    return -ei  # Negative for minimization

result = minimize(
    expected_improvement, 
    X_f8[-1],  # Start from Week 7
    method='L-BFGS-B',
    bounds=[(0.05, 0.30)]*8
)

week8_f8 = result.x
predicted = gp_f8.predict([week8_f8])[0]
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*8)
gp_f8 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=True)
gp_f8.fit(X_f8, y_f8)

best_y = np.max(y_f8)

def expected_improvement(x):
    mu, std = gp_f8.predict([x], return_std=True)
    mu = mu[0]
    std = std[0]
    if std == 0:
        return 0
    improvement = mu - best_y - 0.01
    Z = improvement / std
    ei = improvement * norm.cdf(Z) + std * norm.pdf(Z)
    return -ei

result = minimize(expected_improvement, X_f8[-1], method='L-BFGS-B',
                 bounds=[(0.05, 0.30)]*8)

week8_f8 = result.x
mu_f8 = gp_f8.predict([week8_f8])[0]

print("\nRESULTS:")
print("-"*80)
print(f"Week 7 input: {X_f8[-1]}")
print(f"EI optimized: {week8_f8}")
print(f"Predicted:    {mu_f8:.6f}")
print(f"\n✓ SELECTED: {week8_f8}")
print(f"  Rationale: Expected Improvement found no better point → repeat Week 7")
print()

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("="*80)
print("WEEK 8 FINAL OUTPUTS")
print("="*80)
print()

week8_all = {
    'F1': week8_f1,
    'F2': week8_f2,
    'F3': week8_f3,
    'F4': week8_f4,
    'F5': week8_f5,
    'F6': week8_f6,
    'F7': week8_f7,
    'F8': week8_f8
}

for func, inp in week8_all.items():
    inp_str = "[" + ", ".join([f"{x:.6f}" for x in inp]) + "]"
    print(f"{func}: {inp_str}")

print()
print("="*80)
print("ML METHODS SUMMARY")
print("="*80)
print()
print("F1: GP + RBF kernel + UCB (optimistic recovery)")
print("F2: GP + RBF kernel (tiny length scale) + Exploitation")
print("F3: GP + Matérn kernel + Exploitation (least negative)")
print("F4: GP + Matérn kernel + Exploitation (least negative)")
print("F5: GP + RBF kernel (small length scale) + L-BFGS-B local search")
print("F6: GP + RBF kernel + Exploitation (least negative)")
print("F7: GP + RBF kernel + L-BFGS-B maximization")
print("F8: GP + RBF kernel (ARD) + Expected Improvement")
print()
print("All models: scikit-learn GaussianProcessRegressor")
print("All trained on 7 weeks of historical data")
print("="*80)

WEEK 8 ML CODE - FUNCTION BY FUNCTION

F1 (2D) - RECOVERY MISSION

CODE:
--------------------------------------------------------------------------------

# Build GP model
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1, 0.1])
gp_f1 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=False
)
gp_f1.fit(X_f1, y_f1)

# Test candidates (explore LOWER than Week 6)
candidates = {
    "Lower both": [0.42, 0.42],
    "Much lower": [0.40, 0.40],
    "Asymmetric 1": [0.43, 0.40],
    "Asymmetric 2": [0.40, 0.43],
}

# Select using Upper Confidence Bound (optimistic for recovery)
best_ucb = -np.inf
for name, cand in candidates.items():
    mu, std = gp_f1.predict([cand], return_std=True)
    ucb = mu[0] + 2*std[0]  # Optimistic
    if ucb > best_ucb:
        best_ucb = ucb
        week8_f1 = cand


RESULTS:
--------------------------------------------------------------------------------
Lower both          : μ=0.008202, σ=0.005218, UCB=

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(



RESULTS:
--------------------------------------------------------------------------------
Near Week 4         : μ=-0.091893, σ=0.000370
Slight variation    : μ=-0.023004, σ=0.062248
Conservative        : μ=-0.095058, σ=0.029569

✓ SELECTED: [0.94 0.96 0.8 ]
  Predicted: -0.023004
  Rationale: Return near Week 4 best, minimize damage

F4 (4D) - DAMAGE CONTROL

CODE:
--------------------------------------------------------------------------------

# Build GP with Matérn kernel
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.2]*4, nu=2.5)
gp_f4 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)
gp_f4.fit(X_f4, y_f4)

# Test candidates near center [0.5, 0.5, 0.5, 0.5]
candidates = {
    "Near center (1)": [0.505, 0.495, 0.500, 0.500],
    "Near center (2)": [0.495, 0.505, 0.500, 0.500],
    "Slight asymmetry": [0.52, 0.48, 0.50, 0.50],
}

# Select least negative
best_val = -np.inf
for name, cand in candidates.items():
 

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 3 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(



RESULTS:
--------------------------------------------------------------------------------
Start 1: [1.0, 0.85, 1.0, 0.98] → [0.85 0.85 1.   0.98] → μ=6157.98
Start 2: [0.995, 0.855, 1.0, 0.975] → [0.85  0.855 1.    0.975] → μ=6157.98
Start 3: [1.0, 0.85, 0.995, 0.98] → [0.85 0.85 1.   0.98] → μ=6157.98
Start 4: [0.998, 0.86, 1.0, 0.97] → [0.85 0.86 1.   0.97] → μ=6157.98

✓ SELECTED: [0.85 0.85 1.   0.98]
  Predicted: 6157.98
  Rationale: L-BFGS-B local search around Week 7 peak

F6 (5D) - DAMAGE CONTROL

CODE:
--------------------------------------------------------------------------------

# Build GP with RBF kernel
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*5)
gp_f6 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)
gp_f6.fit(X_f6, y_f6)

# Test candidates near Week 5 best
candidates = {
    "Near Week 5 (1)": [0.25, 0.17, 0.50, 0.49, 0.42],
    "Near Week 5 (2)": [0.27, 0.19, 0.51, 0.47, 0.40],
    "Dim 5 m

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(



RESULTS:
--------------------------------------------------------------------------------
Near Week 5 (1)     : μ=-1.091878, σ=0.004109
Near Week 5 (2)     : μ=-1.092611, σ=0.003665
Dim 5 moderate      : μ=-1.099270, σ=0.017333

✓ SELECTED: [0.25 0.17 0.5  0.49 0.42]
  Predicted: -1.091878
  Rationale: Return near Week 5 best (Dim 5 push failed)

F7 (6D) - EXPLOIT SUCCESS

CODE:
--------------------------------------------------------------------------------

# Build GP with RBF kernel
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1]*6)
gp_f7 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f7.fit(X_f7, y_f7)

# Local optimization (maximize mean)
def neg_pred(x):
    return -gp_f7.predict([x])[0]

result = minimize(
    neg_pred, 
    X_f7[-1],  # Start from Week 7
    method='L-BFGS-B',
    bounds=[(0.0, 0.1), (0.4, 0.5), (0.2, 0.3), 
            (0.1, 0.2), (0.3, 0.4), (0.7, 0.8)]
)

week8_f7 = result.x
predic

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 5 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 7 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [3]:
"""
WEEK 8 - PURE ML OPTIMIZATION (NO MANUAL CANDIDATES)
Let the GP models automatically find optimal points using acquisition functions
"""

import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm

print("="*80)
print("WEEK 8 - PURE ML OPTIMIZATION (NO CANDIDATES)")
print("All points found automatically by optimization algorithms")
print("="*80)
print()

# ============================================================================
# F1 (2D) - UCB OPTIMIZATION
# ============================================================================
print("="*80)
print("F1 (2D) - UPPER CONFIDENCE BOUND OPTIMIZATION")
print("="*80)
print()

X_f1 = np.array([
    [0.10, 0.10],
    [0.12, 0.08],
    [0.21, 0.11],
    [0.14, 0.14],
    [0.08, 0.08],
    [0.45, 0.45],   # Week 6: 0.0128
    [0.48, 0.48],   # Week 7: 0.000008
])
y_f1 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0128, 0.000008])

print("CODE:")
print("-"*80)
print("""
# Build GP
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1, 0.1])
gp_f1 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15, 
                                 alpha=1e-6, normalize_y=False)
gp_f1.fit(X_f1, y_f1)

# Define UCB acquisition function (optimistic for recovery)
def negative_ucb(x, gp, kappa=2.0):
    mu, std = gp.predict([x], return_std=True)
    ucb = mu[0] + kappa * std[0]
    return -ucb  # Negative because we minimize

# Optimize UCB with multiple random starts
bounds = [(0.0, 1.0), (0.0, 1.0)]
best_ucb_val = -np.inf
best_x = None

for i in range(10):  # 10 random starts
    x0 = np.random.uniform(0.0, 1.0, size=2)
    result = minimize(
        lambda x: negative_ucb(x, gp_f1, kappa=2.0),
        x0,
        method='L-BFGS-B',
        bounds=bounds
    )
    ucb_val = -result.fun
    if ucb_val > best_ucb_val:
        best_ucb_val = ucb_val
        best_x = result.x

week8_f1 = best_x
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1, 0.1])
gp_f1 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=False)
gp_f1.fit(X_f1, y_f1)

def negative_ucb(x, gp, kappa=2.0):
    mu, std = gp.predict([x], return_std=True)
    ucb = mu[0] + kappa * std[0]
    return -ucb

bounds_f1 = [(0.0, 1.0), (0.0, 1.0)]
best_ucb_val = -np.inf
best_x = None

print("\nOPTIMIZATION RESULTS:")
print("-"*80)
for i in range(10):
    x0 = np.random.uniform(0.0, 1.0, size=2)
    result = minimize(
        lambda x: negative_ucb(x, gp_f1, kappa=2.0),
        x0,
        method='L-BFGS-B',
        bounds=bounds_f1
    )
    ucb_val = -result.fun
    mu, std = gp_f1.predict([result.x], return_std=True)
    print(f"Start {i+1}: x0={x0} → optimized={result.x} → UCB={ucb_val:.6f}")
    if ucb_val > best_ucb_val:
        best_ucb_val = ucb_val
        best_x = result.x

week8_f1 = best_x
mu_f1, std_f1 = gp_f1.predict([week8_f1], return_std=True)
print(f"\n✓ BEST FOUND: {week8_f1}")
print(f"  UCB: {best_ucb_val:.6f} (μ={mu_f1[0]:.6f}, σ={std_f1[0]:.6f})")
print()

# ============================================================================
# F2 (2D) - EXPLOITATION (MAXIMIZE MEAN)
# ============================================================================
print("="*80)
print("F2 (2D) - PURE EXPLOITATION (MAXIMIZE MEAN)")
print("="*80)
print()

X_f2 = np.array([
    [0.10, 0.10],
    [0.12, 0.08],
    [0.21, 0.11],
    [0.14, 0.14],
    [0.08, 0.08],
    [0.111, 0.100],  # Week 6: 0.1300 BEST
    [0.11, 0.10],    # Week 7: 0.0468
])
y_f2 = np.array([0.0892, 0.0705, 0.0295, 0.0150, 0.0463, 0.1300, 0.0468])

print("CODE:")
print("-"*80)
print("""
# Build GP with small length scale (narrow peak)
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.02, 0.02])
gp_f2 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-7, normalize_y=True)
gp_f2.fit(X_f2, y_f2)

# Define negative mean (for minimization)
def negative_mean(x, gp):
    mu = gp.predict([x])[0]
    return -mu

# Optimize with multiple starts focused near Week 6
bounds = [(0.08, 0.15), (0.08, 0.12)]  # Narrow search near peak
best_mean_val = -np.inf
best_x = None

for i in range(20):  # More starts for narrow peak
    # Start near Week 6 peak
    x0 = np.array([0.111, 0.100]) + np.random.normal(0, 0.01, size=2)
    x0 = np.clip(x0, [0.08, 0.08], [0.15, 0.12])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f2),
        x0,
        method='L-BFGS-B',
        bounds=bounds
    )
    mean_val = -result.fun
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f2 = best_x
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.02, 0.02])
gp_f2 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-7, normalize_y=True)
gp_f2.fit(X_f2, y_f2)

def negative_mean(x, gp):
    mu = gp.predict([x])[0]
    return -mu

bounds_f2 = [(0.08, 0.15), (0.08, 0.12)]
best_mean_val = -np.inf
best_x = None

print("\nOPTIMIZATION RESULTS:")
print("-"*80)
for i in range(20):
    x0 = np.array([0.111, 0.100]) + np.random.normal(0, 0.01, size=2)
    x0 = np.clip(x0, [0.08, 0.08], [0.15, 0.12])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f2),
        x0,
        method='L-BFGS-B',
        bounds=bounds_f2
    )
    mean_val = -result.fun
    if i < 5:  # Show first 5
        print(f"Start {i+1}: x0={x0} → optimized={result.x} → μ={mean_val:.6f}")
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f2 = best_x
print(f"... (15 more starts)")
print(f"\n✓ BEST FOUND: {week8_f2}")
print(f"  Predicted: {best_mean_val:.6f}")
print()

# ============================================================================
# F3 (3D) - EXPLOITATION (MAXIMIZE MEAN FOR NEGATIVES)
# ============================================================================
print("="*80)
print("F3 (3D) - EXPLOITATION (LEAST NEGATIVE)")
print("="*80)
print()

X_f3 = np.array([
    [0.80, 0.80, 0.80],
    [0.95, 0.95, 0.95],
    [0.98, 0.99, 0.87],
    [0.948885, 0.965632, 0.808397],  # Week 4: -0.0786 BEST
    [1.01, 1.01, 0.82],
    [0.928, 0.832, 0.004],
    [0.99, 0.99, 0.99],  # Week 7: -0.427
])
y_f3 = np.array([-0.1055, -0.0919, -0.0856, -0.0786, -1.1543, -0.1161, -0.427251295])

print("CODE:")
print("-"*80)
print("""
# Build GP with Matérn kernel
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.3, 0.3, 0.3], nu=2.5)
gp_f3 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f3.fit(X_f3, y_f3)

# Maximize mean (least negative)
def negative_mean(x, gp):
    mu = gp.predict([x])[0]
    return -mu

# Optimize near Week 4 best region
bounds = [(0.85, 1.0), (0.85, 1.0), (0.70, 0.90)]
best_mean_val = -np.inf
best_x = None

for i in range(15):
    # Start near Week 4
    x0 = np.array([0.95, 0.97, 0.81]) + np.random.normal(0, 0.05, size=3)
    x0 = np.clip(x0, [0.85, 0.85, 0.70], [1.0, 1.0, 0.90])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f3),
        x0,
        method='L-BFGS-B',
        bounds=bounds
    )
    mean_val = -result.fun
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f3 = best_x
""")

# Actually run it
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.3, 0.3, 0.3], nu=2.5)
gp_f3 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f3.fit(X_f3, y_f3)

bounds_f3 = [(0.85, 1.0), (0.85, 1.0), (0.70, 0.90)]
best_mean_val = -np.inf
best_x = None

print("\nOPTIMIZATION RESULTS:")
print("-"*80)
for i in range(15):
    x0 = np.array([0.95, 0.97, 0.81]) + np.random.normal(0, 0.05, size=3)
    x0 = np.clip(x0, [0.85, 0.85, 0.70], [1.0, 1.0, 0.90])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f3),
        x0,
        method='L-BFGS-B',
        bounds=bounds_f3
    )
    mean_val = -result.fun
    if i < 5:
        print(f"Start {i+1}: optimized={result.x} → μ={mean_val:.6f}")
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f3 = best_x
print(f"... (10 more starts)")
print(f"\n✓ BEST FOUND: {week8_f3}")
print(f"  Predicted: {best_mean_val:.6f}")
print()

# ============================================================================
# F4 (4D) - EXPLOITATION NEAR CENTER
# ============================================================================
print("="*80)
print("F4 (4D) - EXPLOITATION (NEAR CENTER)")
print("="*80)
print()

X_f4 = np.array([
    [0.5, 0.5, 0.5, 0.5],  # Week 1: -3.986 BEST
    [0.3, 0.3, 0.3, 0.3],
    [0.44, 0.29, 0.35, 1.25],
    [0.51, 0.60, 0.57, 0.01],
    [0.66, 0.30, 0.30, 0.36],
    [0.2, 0.2, 0.95, 0.4],
    [0.65, 0.65, 0.65, 0.65],  # Week 7: -15.16
])
y_f4 = np.array([-3.986, -4.306, -30.129, -12.492, -7.262, -19.009, -15.158011980])

print("CODE:")
print("-"*80)
print("""
# Build GP
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.2]*4, nu=2.5)
gp_f4 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f4.fit(X_f4, y_f4)

# Maximize mean (least negative)
def negative_mean(x, gp):
    mu = gp.predict([x])[0]
    return -mu

# Focus near center
bounds = [(0.40, 0.60), (0.40, 0.60), (0.40, 0.60), (0.40, 0.60)]
best_mean_val = -np.inf
best_x = None

for i in range(15):
    # Start near center [0.5, 0.5, 0.5, 0.5]
    x0 = np.array([0.5, 0.5, 0.5, 0.5]) + np.random.normal(0, 0.05, size=4)
    x0 = np.clip(x0, 0.40, 0.60)
    
    result = minimize(
        lambda x: negative_mean(x, gp_f4),
        x0,
        method='L-BFGS-B',
        bounds=bounds
    )
    mean_val = -result.fun
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f4 = best_x
""")

# Actually run it
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.2]*4, nu=2.5)
gp_f4 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f4.fit(X_f4, y_f4)

bounds_f4 = [(0.40, 0.60), (0.40, 0.60), (0.40, 0.60), (0.40, 0.60)]
best_mean_val = -np.inf
best_x = None

print("\nOPTIMIZATION RESULTS:")
print("-"*80)
for i in range(15):
    x0 = np.array([0.5, 0.5, 0.5, 0.5]) + np.random.normal(0, 0.05, size=4)
    x0 = np.clip(x0, 0.40, 0.60)
    
    result = minimize(
        lambda x: negative_mean(x, gp_f4),
        x0,
        method='L-BFGS-B',
        bounds=bounds_f4
    )
    mean_val = -result.fun
    if i < 5:
        print(f"Start {i+1}: optimized={result.x} → μ={mean_val:.6f}")
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f4 = best_x
print(f"... (10 more starts)")
print(f"\n✓ BEST FOUND: {week8_f4}")
print(f"  Predicted: {best_mean_val:.6f}")
print()

# ============================================================================
# F5 (4D) - LOCAL SEARCH (MAXIMIZE MEAN)
# ============================================================================
print("="*80)
print("F5 (4D) - LOCAL SEARCH (MAXIMIZE MEAN)")
print("="*80)
print()

X_f5 = np.array([
    [0.30, 0.30, 0.30, 0.30],
    [0.28, 0.32, 0.30, 0.29],
    [0.344822, 0.264687, 0.374156, 0.203902],
    [0.196828, 0.320017, 0.300, 0.289958],
    [0.99, 0.90, 0.98, 0.93],  # Week 5: 5549
    [0.985, 0.905, 0.975, 0.925],  # Week 6: 5399
    [1.0, 0.853, 1.0, 0.977],  # Week 7: 6158 NEW BEST!
])
y_f5 = np.array([136.85, 137.29, 131.78, 140.74, 5549.45, 5398.58, 6158.076608789])

print("CODE:")
print("-"*80)
print("""
# Build GP with small length scale (sharp peak)
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.05, 0.05, 0.05, 0.05])
gp_f5 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=True)
gp_f5.fit(X_f5, y_f5)

# Maximize mean prediction
def negative_mean(x, gp):
    mu = gp.predict([x])[0]
    return -mu

# Local search near Week 7 peak
bounds = [(0.85, 1.0), (0.80, 0.90), (0.95, 1.0), (0.95, 1.0)]
best_mean_val = -np.inf
best_x = None

for i in range(20):  # Many starts for sharp peak
    # Start near Week 7
    x0 = np.array([1.0, 0.853, 1.0, 0.977]) + np.random.normal(0, 0.02, size=4)
    x0 = np.clip(x0, [0.85, 0.80, 0.95, 0.95], [1.0, 0.90, 1.0, 1.0])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f5),
        x0,
        method='L-BFGS-B',
        bounds=bounds
    )
    mean_val = -result.fun
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f5 = best_x
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.05, 0.05, 0.05, 0.05])
gp_f5 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=True)
gp_f5.fit(X_f5, y_f5)

bounds_f5 = [(0.85, 1.0), (0.80, 0.90), (0.95, 1.0), (0.95, 1.0)]
best_mean_val = -np.inf
best_x = None

print("\nOPTIMIZATION RESULTS:")
print("-"*80)
for i in range(20):
    x0 = np.array([1.0, 0.853, 1.0, 0.977]) + np.random.normal(0, 0.02, size=4)
    x0 = np.clip(x0, [0.85, 0.80, 0.95, 0.95], [1.0, 0.90, 1.0, 1.0])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f5),
        x0,
        method='L-BFGS-B',
        bounds=bounds_f5
    )
    mean_val = -result.fun
    if i < 5:
        print(f"Start {i+1}: optimized={result.x} → μ={mean_val:.2f}")
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f5 = best_x
print(f"... (15 more starts)")
print(f"\n✓ BEST FOUND: {week8_f5}")
print(f"  Predicted: {best_mean_val:.2f}")
print()

# ============================================================================
# F6 (5D) - EXPLOITATION NEAR WEEK 5
# ============================================================================
print("="*80)
print("F6 (5D) - EXPLOITATION (NEAR WEEK 5 BEST)")
print("="*80)
print()

X_f6 = np.array([
    [0.75, 0.75, 0.75, 0.75, 0.75],
    [0.3, 0.3, 0.3, 0.3, 0.3],
    [0.49, 0.02, 0.45, 0.40, 0.32],
    [0.69, 0.001, 0.04, 0.001, 0.001],
    [0.26, 0.18, 0.50, 0.48, 0.41],  # Week 5: -1.092 BEST
    [0.1, 0.1, 0.7, 0.7, 0.6],
    [0.15, 0.15, 0.50, 0.50, 0.70],  # Week 7: -1.552
])
y_f6 = np.array([-1.521, -1.139, -1.123, -2.067, -1.092, -1.231, -1.551724443])

print("CODE:")
print("-"*80)
print("""
# Build GP
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*5)
gp_f6 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f6.fit(X_f6, y_f6)

# Maximize mean (least negative)
def negative_mean(x, gp):
    mu = gp.predict([x])[0]
    return -mu

# Focus near Week 5 best
bounds = [(0.15, 0.35), (0.10, 0.25), (0.40, 0.60), (0.40, 0.55), (0.35, 0.50)]
best_mean_val = -np.inf
best_x = None

for i in range(15):
    # Start near Week 5
    x0 = np.array([0.26, 0.18, 0.50, 0.48, 0.41]) + np.random.normal(0, 0.05, size=5)
    x0 = np.clip(x0, [0.15, 0.10, 0.40, 0.40, 0.35], [0.35, 0.25, 0.60, 0.55, 0.50])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f6),
        x0,
        method='L-BFGS-B',
        bounds=bounds
    )
    mean_val = -result.fun
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f6 = best_x
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*5)
gp_f6 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f6.fit(X_f6, y_f6)

bounds_f6 = [(0.15, 0.35), (0.10, 0.25), (0.40, 0.60), (0.40, 0.55), (0.35, 0.50)]
best_mean_val = -np.inf
best_x = None

print("\nOPTIMIZATION RESULTS:")
print("-"*80)
for i in range(15):
    x0 = np.array([0.26, 0.18, 0.50, 0.48, 0.41]) + np.random.normal(0, 0.05, size=5)
    x0 = np.clip(x0, [0.15, 0.10, 0.40, 0.40, 0.35], [0.35, 0.25, 0.60, 0.55, 0.50])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f6),
        x0,
        method='L-BFGS-B',
        bounds=bounds_f6
    )
    mean_val = -result.fun
    if i < 5:
        print(f"Start {i+1}: optimized={result.x} → μ={mean_val:.6f}")
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f6 = best_x
print(f"... (10 more starts)")
print(f"\n✓ BEST FOUND: {week8_f6}")
print(f"  Predicted: {best_mean_val:.6f}")
print()

# ============================================================================
# F7 (6D) - EXPLOITATION (MAXIMIZE MEAN)
# ============================================================================
print("="*80)
print("F7 (6D) - EXPLOITATION (MAXIMIZE MEAN)")
print("="*80)
print()

X_f7 = np.array([
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
    [0.21, 0.19, 0.21, 0.19, 0.17, 0.19],
    [0.08, 0.32, 0.15, 0.28, 0.41, 0.27],
    [0.05, 0.50, 0.25, 0.20, 0.15, 0.85],
    [0.06, 0.48, 0.25, 0.20, 0.40, 0.75],
    [0.038, 0.462, 0.239, 0.171, 0.378, 0.734],  # Week 7: 1.478
])
y_f7 = np.array([0.000034, 0.408, 0.347, 0.568, 0.836, 1.435, 1.478289390])

print("CODE:")
print("-"*80)
print("""
# Build GP
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1]*6)
gp_f7 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=True)
gp_f7.fit(X_f7, y_f7)

# Maximize mean
def negative_mean(x, gp):
    mu = gp.predict([x])[0]
    return -mu

# Local optimization near Week 7
bounds = [(0.0, 0.1), (0.4, 0.5), (0.2, 0.3), (0.1, 0.2), (0.3, 0.4), (0.7, 0.8)]
best_mean_val = -np.inf
best_x = None

for i in range(15):
    # Start near Week 7
    x0 = np.array([0.038, 0.462, 0.239, 0.171, 0.378, 0.734]) + np.random.normal(0, 0.02, size=6)
    x0 = np.clip(x0, [0.0, 0.4, 0.2, 0.1, 0.3, 0.7], [0.1, 0.5, 0.3, 0.2, 0.4, 0.8])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f7),
        x0,
        method='L-BFGS-B',
        bounds=bounds
    )
    mean_val = -result.fun
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f7 = best_x
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1]*6)
gp_f7 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=True)
gp_f7.fit(X_f7, y_f7)

bounds_f7 = [(0.0, 0.1), (0.4, 0.5), (0.2, 0.3), (0.1, 0.2), (0.3, 0.4), (0.7, 0.8)]
best_mean_val = -np.inf
best_x = None

print("\nOPTIMIZATION RESULTS:")
print("-"*80)
for i in range(15):
    x0 = np.array([0.038, 0.462, 0.239, 0.171, 0.378, 0.734]) + np.random.normal(0, 0.02, size=6)
    x0 = np.clip(x0, [0.0, 0.4, 0.2, 0.1, 0.3, 0.7], [0.1, 0.5, 0.3, 0.2, 0.4, 0.8])
    
    result = minimize(
        lambda x: negative_mean(x, gp_f7),
        x0,
        method='L-BFGS-B',
        bounds=bounds_f7
    )
    mean_val = -result.fun
    if i < 5:
        print(f"Start {i+1}: optimized={result.x} → μ={mean_val:.6f}")
    if mean_val > best_mean_val:
        best_mean_val = mean_val
        best_x = result.x

week8_f7 = best_x
print(f"... (10 more starts)")
print(f"\n✓ BEST FOUND: {week8_f7}")
print(f"  Predicted: {best_mean_val:.6f}")
print()

# ============================================================================
# F8 (8D) - EXPECTED IMPROVEMENT
# ============================================================================
print("="*80)
print("F8 (8D) - EXPECTED IMPROVEMENT")
print("="*80)
print()

X_f8 = np.array([
    [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    [0.12, 0.09, 0.11, 0.10, 0.08, 0.13, 0.11, 0.09],
    [0.29, 0.25, 0.02, 0.29, 0.14, 0.22, 0.25, 0.30],
    [1.0, 0.001, 1.0, 0.001, 0.001, 1.0, 1.0, 0.001],
    [0.05, 0.25, 0.25, 0.25, 0.25, 0.25, 0.05, 0.05],
    [0.18, 0.15, 0.20, 0.15, 0.25, 0.15, 0.15, 0.18],
    [0.177, 0.194, 0.170, 0.194, 0.294, 0.143, 0.109, 0.208],  # Week 7: 9.692
])
y_f8 = np.array([9.542, 9.554, 9.548, 4.180, 9.643, 9.676, 9.692074600])

print("CODE:")
print("-"*80)
print("""
# Build GP with ARD
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*8)
gp_f8 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=True)
gp_f8.fit(X_f8, y_f8)

# Expected Improvement
best_y = np.max(y_f8)

def negative_ei(x, gp, best_y, xi=0.01):
    mu, std = gp.predict([x], return_std=True)
    mu = mu[0]
    std = std[0]
    if std == 0:
        return 0
    improvement = mu - best_y - xi
    Z = improvement / std
    ei = improvement * norm.cdf(Z) + std * norm.pdf(Z)
    return -ei

# Optimize with multiple random starts
bounds = [(0.05, 0.30)]*8
best_ei_val = -np.inf
best_x = None

for i in range(20):  # More starts for 8D
    x0 = np.random.uniform(0.05, 0.30, size=8)
    
    result = minimize(
        lambda x: negative_ei(x, gp_f8, best_y),
        x0,
        method='L-BFGS-B',
        bounds=bounds
    )
    ei_val = -result.fun
    if ei_val > best_ei_val:
        best_ei_val = ei_val
        best_x = result.x

week8_f8 = best_x
""")

# Actually run it
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*8)
gp_f8 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=True)
gp_f8.fit(X_f8, y_f8)

best_y = np.max(y_f8)

def negative_ei(x, gp, best_y, xi=0.01):
    mu, std = gp.predict([x], return_std=True)
    mu = mu[0]
    std = std[0]
    if std == 0:
        return 0
    improvement = mu - best_y - xi
    Z = improvement / std
    ei = improvement * norm.cdf(Z) + std * norm.pdf(Z)
    return -ei

bounds_f8 = [(0.05, 0.30)]*8
best_ei_val = -np.inf
best_x = None

print("\nOPTIMIZATION RESULTS:")
print("-"*80)
for i in range(20):
    x0 = np.random.uniform(0.05, 0.30, size=8)
    
    result = minimize(
        lambda x: negative_ei(x, gp_f8, best_y),
        x0,
        method='L-BFGS-B',
        bounds=bounds_f8
    )
    ei_val = -result.fun
    if i < 5:
        mu_pred = gp_f8.predict([result.x])[0]
        print(f"Start {i+1}: EI={ei_val:.6f}, μ={mu_pred:.3f}")
    if ei_val > best_ei_val:
        best_ei_val = ei_val
        best_x = result.x

week8_f8 = best_x
mu_f8 = gp_f8.predict([week8_f8])[0]
print(f"... (15 more starts)")
print(f"\n✓ BEST FOUND (by EI): {week8_f8}")
print(f"  EI: {best_ei_val:.6f}")
print(f"  Predicted: {mu_f8:.3f}")
print()

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("="*80)
print("WEEK 8 FINAL OUTPUTS (PURE ML OPTIMIZATION)")
print("="*80)
print()

week8_all = {
    'F1': week8_f1,
    'F2': week8_f2,
    'F3': week8_f3,
    'F4': week8_f4,
    'F5': week8_f5,
    'F6': week8_f6,
    'F7': week8_f7,
    'F8': week8_f8
}

for func, inp in week8_all.items():
    inp_str = "[" + ", ".join([f"{x:.6f}" for x in inp]) + "]"
    print(f"{func}: {inp_str}")

print()
print("="*80)
print("OPTIMIZATION METHODS SUMMARY")
print("="*80)
print()
print("F1: UCB (Upper Confidence Bound) - Optimistic recovery")
print("    • Multiple random starts (10)")
print("    • kappa=2.0 for exploration")
print()
print("F2: Exploitation (Maximize Mean) - Return to peak")
print("    • Narrow bounds near Week 6")
print("    • 20 starts near [0.111, 0.100]")
print()
print("F3: Exploitation (Maximize Mean) - Minimize damage")
print("    • 15 starts near Week 4 best")
print("    • Matérn kernel for sharp boundaries")
print()
print("F4: Exploitation (Maximize Mean) - Stay at center")
print("    • Bounded [0.4, 0.6] on all dims")
print("    • 15 starts near [0.5, 0.5, 0.5, 0.5]")
print()
print("F5: Exploitation (Maximize Mean) - Sharp peak navigation")
print("    • Small length scale (0.05)")
print("    • 20 starts near Week 7 best")
print("    • Tight bounds for local search")
print()
print("F6: Exploitation (Maximize Mean) - Return to Week 5")
print("    • 15 starts near Week 5 best")
print("    • Bounded near known good region")
print()
print("F7: Exploitation (Maximize Mean) - Continue success")
print("    • 15 starts near Week 7")
print("    • Dimension-specific bounds")
print()
print("F8: Expected Improvement - Balanced exploration/exploitation")
print("    • 20 random starts (8D requires more)")
print("    • xi=0.01 for balanced trade-off")
print()
print("ALL OPTIMIZATIONS:")
print("  • Algorithm: L-BFGS-B (gradient-based)")
print("  • Multiple random restarts to avoid local optima")
print("  • Strategic starting points near known good regions")
print("  • Bounded search spaces based on domain knowledge")
print()
print("="*80)
print("NO MANUAL CANDIDATES - ALL POINTS FOUND AUTOMATICALLY!")
print("="*80)

WEEK 8 - PURE ML OPTIMIZATION (NO CANDIDATES)
All points found automatically by optimization algorithms

F1 (2D) - UPPER CONFIDENCE BOUND OPTIMIZATION

CODE:
--------------------------------------------------------------------------------

# Build GP
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1, 0.1])
gp_f1 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15, 
                                 alpha=1e-6, normalize_y=False)
gp_f1.fit(X_f1, y_f1)

# Define UCB acquisition function (optimistic for recovery)
def negative_ucb(x, gp, kappa=2.0):
    mu, std = gp.predict([x], return_std=True)
    ucb = mu[0] + kappa * std[0]
    return -ucb  # Negative because we minimize

# Optimize UCB with multiple random starts
bounds = [(0.0, 1.0), (0.0, 1.0)]
best_ucb_val = -np.inf
best_x = None

for i in range(10):  # 10 random starts
    x0 = np.random.uniform(0.0, 1.0, size=2)
    result = minimize(
        lambda x: negative_ucb(x, gp_f1, kappa=2.0),
        x0,
        method=

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(



OPTIMIZATION RESULTS:
--------------------------------------------------------------------------------
Start 1: x0=[0.16477798 0.78692378] → optimized=[0.16477798 0.78692378] → UCB=0.012623
Start 2: x0=[0.63396476 0.83814689] → optimized=[0.63396476 0.83814689] → UCB=0.012623
Start 3: x0=[0.01206098 0.03029727] → optimized=[0.01206098 0.        ] → UCB=0.012622
Start 4: x0=[0.98953368 0.30645649] → optimized=[0.98953368 0.30645649] → UCB=0.012623
Start 5: x0=[0.99988573 0.42197699] → optimized=[0.99988573 0.42761556] → UCB=0.019174
Start 6: x0=[0.20540786 0.19698545] → optimized=[0.20540786 0.23528136] → UCB=0.012623
Start 7: x0=[0.71216216 0.63144554] → optimized=[0.71216216 0.63144554] → UCB=0.012623
Start 8: x0=[0.80296272 0.09791326] → optimized=[0.80296272 0.        ] → UCB=0.012622
Start 9: x0=[0.30764532 0.30118296] → optimized=[0.30764532 0.30118296] → UCB=0.012623
Start 10: x0=[0.89210222 0.47714706] → optimized=[0.89210222 0.        ] → UCB=0.012622

✓ BEST FOUND: [0.9998857

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(



OPTIMIZATION RESULTS:
--------------------------------------------------------------------------------
Start 1: x0=[0.12486447 0.09090973] → optimized=[0.12486447 0.09090973] → μ=0.061043
Start 2: x0=[0.10468949 0.08687212] → optimized=[0.10468949 0.08687212] → μ=0.061043
Start 3: x0=[0.12641917 0.11182572] → optimized=[0.12641917 0.11182572] → μ=0.061043
Start 4: x0=[0.10151125 0.10041022] → optimized=[0.10151125 0.10041022] → μ=0.061043
Start 5: x0=[0.11860974 0.1037788 ] → optimized=[0.11860974 0.1037788 ] → μ=0.061043
... (15 more starts)

✓ BEST FOUND: [0.12486447 0.09090973]
  Predicted: 0.061043

F3 (3D) - EXPLOITATION (LEAST NEGATIVE)

CODE:
--------------------------------------------------------------------------------

# Build GP with Matérn kernel
kernel = ConstantKernel(1.0) * Matern(length_scale=[0.3, 0.3, 0.3], nu=2.5)
gp_f3 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=15,
                                 alpha=1e-6, normalize_y=True)
gp_f3.fit(X_f3, y

...


OPTIMIZATION RESULTS:
--------------------------------------------------------------------------------
Start 1: optimized=[0.25599627 0.19441245 0.51380405 0.55       0.42444168] → μ=-1.088549
Start 2: optimized=[0.23757026 0.17697872 0.51072187 0.55       0.42444097] → μ=-1.088549
Start 3: optimized=[0.19738037 0.14921338 0.6        0.55       0.42444057] → μ=-1.088549
Start 4: optimized=[0.24748647 0.23195114 0.57119851 0.55       0.42444099] → μ=-1.088549
Start 5: optimized=[0.25082743 0.1168503  0.47779522 0.55       0.4244407 ] → μ=-1.088549
... (10 more starts)

✓ BEST FOUND: [0.24748647 0.23195114 0.57119851 0.55       0.42444099]
  Predicted: -1.088549

F7 (6D) - EXPLOITATION (MAXIMIZE MEAN)

CODE:
--------------------------------------------------------------------------------

# Build GP
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.1]*6)
gp_f7 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,
                                 alpha=1e-6, normalize_y=Tru

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 3 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(



OPTIMIZATION RESULTS:
--------------------------------------------------------------------------------
Start 1: optimized=[0.04217691 0.45733899 0.2363962  0.18285509 0.34582584 0.70544363] → μ=1.508511
Start 2: optimized=[0.00890851 0.41567017 0.2181247  0.17073127 0.39392079 0.70544363] → μ=1.508511
Start 3: optimized=[0.07129031 0.44818837 0.21764998 0.15098167 0.39999998 0.70544363] → μ=1.508511
Start 4: optimized=[0.0314245  0.44498279 0.24447907 0.18298607 0.36657241 0.70544363] → μ=1.508511
Start 5: optimized=[0.05430096 0.4534588  0.22543279 0.15589697 0.38398658 0.70544363] → μ=1.508511
... (10 more starts)

✓ BEST FOUND: [0.00890851 0.41567017 0.2181247  0.17073127 0.39392079 0.70544363]
  Predicted: 1.508511

F8 (8D) - EXPECTED IMPROVEMENT

CODE:
--------------------------------------------------------------------------------

# Build GP with ARD
kernel = ConstantKernel(1.0) * RBF(length_scale=[0.2]*8)
gp_f8 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 5 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 7 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(



OPTIMIZATION RESULTS:
--------------------------------------------------------------------------------
Start 1: EI=0.000000, μ=9.551
Start 2: EI=0.000000, μ=9.545
Start 3: EI=0.000000, μ=9.354
Start 4: EI=0.000000, μ=9.384
Start 5: EI=0.139866, μ=9.842
... (15 more starts)

✓ BEST FOUND (by EI): [0.21473545 0.28957844 0.3        0.3        0.3        0.0856279
 0.14461032 0.15624306]
  EI: 0.139866
  Predicted: 9.842

WEEK 8 FINAL OUTPUTS (PURE ML OPTIMIZATION)

F1: [0.999886, 0.427616]
F2: [0.124864, 0.090910]
F3: [0.940740, 1.000000, 0.742901]
F4: [0.400000, 0.472917, 0.455962, 0.415896]
F5: [0.850000, 0.825374, 1.000000, 0.984237]
F6: [0.247486, 0.231951, 0.571199, 0.550000, 0.424441]
F7: [0.008909, 0.415670, 0.218125, 0.170731, 0.393921, 0.705444]
F8: [0.214735, 0.289578, 0.300000, 0.300000, 0.300000, 0.085628, 0.144610, 0.156243]

OPTIMIZATION METHODS SUMMARY

F1: UCB (Upper Confidence Bound) - Optimistic recovery
    • Multiple random starts (10)
    • kappa=2.0 for exploration
